In [2]:
from IPython.display import display, clear_output
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import time

# Load your pre-trained emotion classifier model
emotion_model = load_model('haaugmentation_model.keras')

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the image from BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            face_roi = cv2.resize(face_roi, (48, 48))
            face_roi = face_roi / 255.0
            face_roi = np.expand_dims(face_roi, axis=0)
            face_roi = np.expand_dims(face_roi, axis=3)

            # Predicting the emotion
            emotion_probabilities = emotion_model.predict(face_roi)
            print("Emotion probabilities:", emotion_probabilities)  # Debug output

            # Assuming the output is a single probability of being 'Happy'
            happy_probability = emotion_probabilities[0][0]
            emotion_text = "Happy" if happy_probability > 0.5 else "Angry"

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emotion_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Convert to PIL Image for display
        im = Image.fromarray(frame)

        # Display image
        display(im)
        clear_output(wait=True)

        time.sleep(0.03)  # Adjust this based on your webcam's framerate capabilities

except KeyboardInterrupt:
    print("Stream stopped.")
    cap.release()

Stream stopped.
